# Call consensus sequences for viral barcodes in each cell
This Python Jupyter notebook corrects/calls consensus viral barcodes in each cell. Viral barcodes are input as UMI count data, assigned to a cell barcode and gene. The notebook uses UMI tools to correct the viral barcodes for each cell; this provides a set of consensus viral barcodes within each cell. The notebo as well as a whitelist of all viral barcodes identified on each segment for each tag.

## Notes about UMI_tools
* Using directional adjacency method. This has [been demonstrated on simulated data](https://cgatoxford.wordpress.com/2015/08/14/unique-molecular-identifiers-the-problem-the-solution-and-the-proof/) to produce a more accurate estimate of true UMI number than other heuristics.
* Sequence must be input as byte. See definition here: https://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal
* The corrected barcode is returned as the first barcode in the group list. See umi_tools API documentation: https://umi-tools.readthedocs.io/en/latest/API.html

Import Python modules: